In [1]:
from PIL import Image
import os

import torch
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms

from sklearn.metrics import classification_report, fbeta_score, matthews_corrcoef
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
# Define hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

In [3]:
# Set the device
device = "cpu"
if torch.cuda.is_available():
    device =  "cuda"
elif torch.backends.mps.is_available():
    device = "mps" # Use M1 Mac GPU
print(device)

cuda


# Extractor classifier dataset

In [4]:
class ExtractorClassifierDataset(Dataset):
    def __init__(self, data, resize_shape=(256, 256)):
        self.data = data
        self.to_tensor = transforms.ToTensor()
        self.resize_image = transforms.Resize(resize_shape, antialias=True)
        self.image_tensors = []
        self.label_tensors = []

        for image, label in tqdm(self.data):
            # Apply transformations to the image and convert to tensor
            image_tensor = self.to_tensor(image)
            image_tensor = self.resize_image(image_tensor)
            image_tensor = image_tensor/255
            self.image_tensors.append(image_tensor)

            # Create one-hot encoded tensor for the label
            label_tensor = torch.zeros(2)
            label_tensor[label] = 1
            self.label_tensors.append(label_tensor)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.image_tensors[idx], self.label_tensors[idx]

data_dir = "../../../datasets/extractor_classifier"

train_list = []
validation_list = []
test_list = []

for class_name in ("relevant", "not_relevant"):
    if class_name == "relevant":
        label = 0
    else:
        label = 1

    for image in os.listdir(f"{data_dir}/train/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/train/{class_name}/{image}"
            train_list.append((Image.open(src_path), label))
            
# Uncomment this part to add augmented data
    #for image in os.listdir(f"{data_dir}/train_data_augmentation/{class_name}"):
        #if image != ".ipynb_checkpoints":
            #src_path = f"{data_dir}/train_data_augmentation/{class_name}/{image}"
            #train_list.append((Image.open(src_path), label))

    for image in os.listdir(f"{data_dir}/validation/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/validation/{class_name}/{image}"
            validation_list.append((Image.open(src_path), label))

    for image in os.listdir(f"{data_dir}/test/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/test/{class_name}/{image}"
            test_list.append((Image.open(src_path), label))

# Define the train/validation/test datasets
train_data = ExtractorClassifierDataset(train_list)
validation_data = ExtractorClassifierDataset(validation_list)
test_data = ExtractorClassifierDataset(test_list)

100%|██████████| 697/697 [00:06<00:00, 111.10it/s]


In [5]:
# Define the dataloaders for each dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)

# Model implementations

In [6]:
from torchvision.models import resnet50

class Resnet50Model(nn.Module):
    def __init__(self, pretrained=False):
        super(Resnet50Model, self).__init__()
        self.resnet_model = resnet50(pretrained=pretrained)
        num_features = self.resnet_model.fc.in_features
        self.resnet_model.fc = nn.Linear(num_features, 2)
        
    def forward(self, x):
        x = self.resnet_model(x)
        return x

In [6]:
from torchvision.models import convnext_base

class ConvNeXtBaseModel(nn.Module):
    def __init__(self, pretrained=False):
        super(ConvNeXtBaseModel, self).__init__()
        self.convnext_model = convnext_base(pretrained=pretrained)
        num_features = self.convnext_model.classifier[2].in_features
        new_classifier_layer = torch.nn.Linear(num_features, 2)
        self.convnext_model.classifier[2] = new_classifier_layer
        
    def forward(self, x):
        x = self.convnext_model(x)
        return x

In [6]:
from torchvision.models import efficientnet_v2_m

class EfficientNetModel(nn.Module):
    def __init__(self, pretrained=False):
        super(EfficientNetModel, self).__init__()
        self.efficientnet_model = efficientnet_v2_m(pretrained=pretrained)
        num_features = self.efficientnet_model.classifier[1].in_features
        new_classifier_layer = torch.nn.Linear(num_features, 2)
        self.efficientnet_model.classifier[1] = new_classifier_layer
        
    def forward(self, x):
        x = self.efficientnet_model(x)
        return x

# Train and validation

In [12]:
def train(model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, epochs, gradient_accumulation=1):
    model.to(device)

    # Define variables to track the model performance
    best_valid_f_beta = 0.0
    train_loss_values = []
    validation_loss_values = []
    train_f_beta_values = []
    validation_f_beta_values = []

    train_true_labels = []
    train_predicted_labels = []

    for epoch in range(epochs):
        train_loss = 0.0
        model.train()
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * images.size(0)
            loss.backward()

            # Use gradient accumulation because large batch sizes lead to out of memory
            if (i + 1) % gradient_accumulation == 0 or (i + 1) == len(train_loader):
                optimizer.step()
                optimizer.zero_grad()
            
            outputs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            train_predicted_labels.extend(predicted.detach().cpu())
            train_true_labels.extend(labels.detach().cpu())

        train_loss = train_loss / len(train_loader.dataset)
        train_loss_values.append(train_loss)

        train_f_beta = fbeta_score(train_true_labels, train_predicted_labels, average="binary", beta=0.5)
        train_f_beta_values.append(train_f_beta)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss:.4f} - Train Fbeta: {train_f_beta:.4f}")

        # Evaluate the model on the validation set
        validation_loss, validation_f_beta = validate_model(model, validation_loader, criterion)
        validation_loss_values.append(validation_loss)
        validation_f_beta_values.append(validation_f_beta)

        # Save the model state if the current validation f beta score is better than the previous best
        if validation_f_beta > best_valid_f_beta:
            best_valid_f_beta = validation_f_beta
            print(f"Updated best model: epoch {epoch+1}")
            torch.save(model.state_dict(), model_path)

    # Load the best model state and evaluate on the test set
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Test model:")
    test_loss, test_f_beta = validate_model(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f} - Test Fbeta: {test_f_beta:.4f}")

    return model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values

def validate_model(model, data_loader, criterion):
    model.to(device)
    model.eval()

    valid_loss = 0.0
    valid_true_labels = []
    valid_predicted_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)

            outputs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            valid_predicted_labels.extend(predicted.detach().cpu())
            valid_true_labels.extend(labels.detach().cpu())

    f_beta_score = fbeta_score(valid_true_labels, valid_predicted_labels, average="binary", beta=0.5)
    mcc_score = matthews_corrcoef(valid_true_labels, valid_predicted_labels)
    valid_loss = valid_loss / len(data_loader.dataset)

    print(f"Valid Loss: {valid_loss:.4f} - Valid Fbeta: {f_beta_score:.4f} - Valid MCC: {mcc_score}")
    print(classification_report(valid_true_labels, valid_predicted_labels))
    return valid_loss, f_beta_score

# ResNet50

In [24]:
resnet_model = Resnet50Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/resnet50_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    resnet_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(resnet_model, validation_loader, criterion)
    print("Test data:")
    validate_model(resnet_model, test_loader, criterion)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(resnet_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50, 4)
    np.save("model_results/resnet50_model_50_train_loss", train_loss_values)
    np.save("model_results/resnet50_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/resnet50_model_50_valid_loss", validation_loss_values)
    np.save("model_results/resnet50_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 77/77 [00:09<00:00,  7.90it/s]


Epoch 1/50 - Train Loss: 0.6004 - Train Fbeta: 0.5850
Valid Loss: 0.7401 - Valid Fbeta: 0.3645 - Valid MCC: -0.2090896788810389
              precision    recall  f1-score   support

           0       0.42      0.12      0.19       218
           1       0.33      0.71      0.45       130

    accuracy                           0.34       348
   macro avg       0.37      0.42      0.32       348
weighted avg       0.38      0.34      0.29       348

Updated best model: epoch 1


100%|██████████| 77/77 [00:09<00:00,  7.95it/s]


...

Epoch 48/50 - Train Loss: 0.0360 - Train Fbeta: 0.9342
Valid Loss: 1.1838 - Valid Fbeta: 0.7642 - Valid MCC: 0.5728947271789466
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       218
           1       0.81      0.62      0.70       130

    accuracy                           0.80       348
   macro avg       0.81      0.77      0.78       348
weighted avg       0.81      0.80      0.80       348



100%|██████████| 77/77 [00:09<00:00,  7.96it/s]


Epoch 49/50 - Train Loss: 0.0515 - Train Fbeta: 0.9350
Valid Loss: 0.7236 - Valid Fbeta: 0.7388 - Valid MCC: 0.5920814958926555
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       218
           1       0.73      0.76      0.75       130

    accuracy                           0.81       348
   macro avg       0.79      0.80      0.80       348
weighted avg       0.81      0.81      0.81       348



100%|██████████| 77/77 [00:09<00:00,  7.92it/s]


Epoch 50/50 - Train Loss: 0.0537 - Train Fbeta: 0.9357
Valid Loss: 0.7876 - Valid Fbeta: 0.8014 - Valid MCC: 0.6440676460736462
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       218
           1       0.83      0.71      0.76       130

    accuracy                           0.84       348
   macro avg       0.83      0.81      0.82       348
weighted avg       0.84      0.84      0.83       348

Test model:
Valid Loss: 0.9274 - Valid Fbeta: 0.8206 - Valid MCC: 0.6339411726828715
              precision    recall  f1-score   support

           0       0.81      0.96      0.88       436
           1       0.89      0.62      0.73       261

    accuracy                           0.83       697
   macro avg       0.85      0.79      0.80       697
weighted avg       0.84      0.83      0.82       697

Test Loss: 0.9274 - Test Fbeta: 0.8206


In [25]:
efficient_net_model = EfficientNetModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficient_net_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/efficient_net50_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    efficient_net_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(efficient_net_model, validation_loader, criterion)
    print("Test data:")
    validate_model(efficient_net_model, test_loader, criterion)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(efficient_net_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50, 4)
    np.save("model_results/efficient_net50_model_50_train_loss", train_loss_values)
    np.save("model_results/efficient_net50_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/efficient_net50_model_50_valid_loss", validation_loss_values)
    np.save("model_results/efficient_net50_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 77/77 [00:19<00:00,  3.97it/s]


Epoch 1/50 - Train Loss: 0.6872 - Train Fbeta: 0.3805


/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

...

Epoch 48/50 - Train Loss: 0.0354 - Train Fbeta: 0.8900
Valid Loss: 0.8271 - Valid Fbeta: 0.8246 - Valid MCC: 0.6759525677730313
              precision    recall  f1-score   support

           0       0.85      0.93      0.89       218
           1       0.85      0.72      0.78       130

    accuracy                           0.85       348
   macro avg       0.85      0.82      0.83       348
weighted avg       0.85      0.85      0.85       348



100%|██████████| 77/77 [00:19<00:00,  3.99it/s]


Epoch 49/50 - Train Loss: 0.0256 - Train Fbeta: 0.8922
Valid Loss: 0.7878 - Valid Fbeta: 0.8305 - Valid MCC: 0.6950766181818684
              precision    recall  f1-score   support

           0       0.86      0.92      0.89       218
           1       0.85      0.75      0.80       130

    accuracy                           0.86       348
   macro avg       0.86      0.84      0.85       348
weighted avg       0.86      0.86      0.86       348



100%|██████████| 77/77 [00:19<00:00,  3.99it/s]


Epoch 50/50 - Train Loss: 0.0415 - Train Fbeta: 0.8942
Valid Loss: 0.9316 - Valid Fbeta: 0.8273 - Valid MCC: 0.6700525638546848
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       218
           1       0.87      0.70      0.77       130

    accuracy                           0.85       348
   macro avg       0.85      0.82      0.83       348
weighted avg       0.85      0.85      0.84       348

Test model:
Valid Loss: 0.4913 - Valid Fbeta: 0.8326 - Valid MCC: 0.668829888509247
              precision    recall  f1-score   support

           0       0.83      0.94      0.89       436
           1       0.88      0.68      0.77       261

    accuracy                           0.85       697
   macro avg       0.86      0.81      0.83       697
weighted avg       0.85      0.85      0.84       697

Test Loss: 0.4913 - Test Fbeta: 0.8326


# ConvNeXt model training

In [9]:
convnext_model = ConvNeXtBaseModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(convnext_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/convnext_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    convnext_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(convnext_model, validation_loader, criterion)
    print("Test data:")
    validate_model(convnext_model, test_loader, criterion)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(convnext_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50, 4)
    np.save("model_results/convnext_model_50_train_loss", train_loss_values)
    np.save("model_results/convnext_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/convnext_model_50_valid_loss", validation_loss_values)
    np.save("model_results/convnext_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 77/77 [00:47<00:00,  1.62it/s]


Epoch 1/50 - Train Loss: 1.0076 - Train Fbeta: 0.3704


/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

...

Epoch 48/50 - Train Loss: 0.2087 - Train Fbeta: 0.7117
Valid Loss: 0.6410 - Valid Fbeta: 0.7852 - Valid MCC: 0.6118121668440281
              precision    recall  f1-score   support

           0       0.82      0.91      0.87       218
           1       0.82      0.67      0.74       130

    accuracy                           0.82       348
   macro avg       0.82      0.79      0.80       348
weighted avg       0.82      0.82      0.82       348

Updated best model: epoch 48


100%|██████████| 77/77 [00:46<00:00,  1.64it/s]


Epoch 49/50 - Train Loss: 0.1669 - Train Fbeta: 0.7175
Valid Loss: 0.6163 - Valid Fbeta: 0.7852 - Valid MCC: 0.6118121668440281
              precision    recall  f1-score   support

           0       0.82      0.91      0.87       218
           1       0.82      0.67      0.74       130

    accuracy                           0.82       348
   macro avg       0.82      0.79      0.80       348
weighted avg       0.82      0.82      0.82       348



100%|██████████| 77/77 [00:46<00:00,  1.65it/s]


Epoch 50/50 - Train Loss: 0.1725 - Train Fbeta: 0.7230
Valid Loss: 0.6389 - Valid Fbeta: 0.7765 - Valid MCC: 0.6127567111500221
              precision    recall  f1-score   support

           0       0.83      0.89      0.86       218
           1       0.80      0.70      0.75       130

    accuracy                           0.82       348
   macro avg       0.82      0.80      0.80       348
weighted avg       0.82      0.82      0.82       348

Test model:
Valid Loss: 0.6288 - Valid Fbeta: 0.7484 - Valid MCC: 0.5617267269212847
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       436
           1       0.78      0.65      0.71       261

    accuracy                           0.80       697
   macro avg       0.79      0.77      0.78       697
weighted avg       0.80      0.80      0.79       697

Test Loss: 0.6288 - Test Fbeta: 0.7484


# Train with augmented data

In [14]:
efficient_net_model_augmented_data = EfficientNetModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficient_net_model_augmented_data.parameters(), lr=LEARNING_RATE)
model_path = "model_results/efficient_net_augmented_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    efficient_net_model_augmented_data.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(efficient_net_model_augmented_data, validation_loader, criterion)
    print("Test data:")
    validate_model(efficient_net_model_augmented_data, test_loader, criterion)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(efficient_net_model_augmented_data, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50, 4)
    np.save("model_results/efficient_net_augmented_model_50_train_loss", train_loss_values)
    np.save("model_results/efficient_net_augmented_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/efficient_net_augmented_model_50_valid_loss", validation_loss_values)
    np.save("model_results/efficient_net_augmented_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 229/229 [00:59<00:00,  3.86it/s]


Epoch 1/50 - Train Loss: 0.6820 - Train Fbeta: 0.2309


/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

...

Epoch 48/50 - Train Loss: 0.0218 - Train Fbeta: 0.9279
Valid Loss: 0.8954 - Valid Fbeta: 0.8247 - Valid MCC: 0.6823172752546119
              precision    recall  f1-score   support

           0       0.86      0.92      0.89       218
           1       0.85      0.74      0.79       130

    accuracy                           0.85       348
   macro avg       0.85      0.83      0.84       348
weighted avg       0.85      0.85      0.85       348



100%|██████████| 229/229 [00:58<00:00,  3.91it/s]


Epoch 49/50 - Train Loss: 0.0190 - Train Fbeta: 0.9292
Valid Loss: 1.0488 - Valid Fbeta: 0.7788 - Valid MCC: 0.642835120668926
              precision    recall  f1-score   support

           0       0.86      0.87      0.87       218
           1       0.78      0.77      0.78       130

    accuracy                           0.83       348
   macro avg       0.82      0.82      0.82       348
weighted avg       0.83      0.83      0.83       348



100%|██████████| 229/229 [00:58<00:00,  3.94it/s]


Epoch 50/50 - Train Loss: 0.0192 - Train Fbeta: 0.9305
Valid Loss: 1.1413 - Valid Fbeta: 0.7740 - Valid MCC: 0.637204583671474
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       218
           1       0.78      0.77      0.77       130

    accuracy                           0.83       348
   macro avg       0.82      0.82      0.82       348
weighted avg       0.83      0.83      0.83       348

Test model:
Valid Loss: 0.9996 - Valid Fbeta: 0.7979 - Valid MCC: 0.6386634151183261
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       436
           1       0.83      0.70      0.76       261

    accuracy                           0.83       697
   macro avg       0.83      0.81      0.82       697
weighted avg       0.83      0.83      0.83       697

Test Loss: 0.9996 - Test Fbeta: 0.7979
